# Estimation Tutorial

In this section, we dive into the topic of model estimation using **pydsge**. Note that for this tutorial we will assume a folder set-up of the form

```
analysis/
├── README.md
├── src/ 
│   ├── estimation.py or .ipynb 
│   └── model.yaml   
├── data/
│   └── example_data
└── output/
```
This is because the estimation creates (intermediate) output results, which we will want to store.

In [21]:
# Just for the tutorial: Setting up example structure
import tempfile
import os
import shutil # For clean-up of temporary directory
from pathlib import Path # For Windows/Unix compatibility

# Temporary output folder
output_path = Path(tempfile.gettempdir(), 'output')
os.makedirs(output_path)

# Parsing and loading the model

Let us first load the relevant packages. Besides the DSGE class we already know from [*getting started*](https://pydsge.readthedocs.io/en/latest/getting_started.html), we also want to import the `emcee` package. This will allow us to later specify the desired updating algorithms for sampling from the posterior distribution - we explain this in more detail below.

In [10]:
import pandas as pd
import numpy as np
import emcee # For specifying updating moves

from pydsge import DSGE, example

In this tutorial, we continue to use the example provided in `pydsge`. Like before, we specify the file paths of the model and the data. Please feel free to check-out both files, but from the previous tutorial you might remember that we're dealing with a five equations New Keynesian model and US quarterly data from 1995 to 2018. 

In [11]:
yaml_file, data_file = example

We again parse the model and load-in the data. What is important is that we also specify a location where the (intermediate) output is stored. Here we assign the output folder, as discussed at the beginning. Note also that we can name the model and write a short description, which is very useful when working with several models.

In [13]:
# Parse the model
mod = DSGE.read(yaml_file)  

# Give it a name
mod.name = 'Rank_tutorial'
mod.description = 'RANK, estimation tutorial'

# Storage location for output
mod.path = output_path

# Load data
df = pd.read_csv(data_file, parse_dates=['date'], index_col=['date'])
df.index.freq = 'Q' # let pandas know that this is quartely data

Remember that since the Great Recession, the Federal Funds Rate has been below the ZLB. That is why, like in [*getting started*](https://pydsge.readthedocs.io/en/latest/getting_started.html), we adjust the observed interest rate, so that the data is within reach of our model.

In [15]:
# adjust elb
zlb = mod.get_par('elb_level')
rate = df['FFR']
df['FFR'] = np.maximum(rate,zlb)

mod.load_data(df, start='1998Q1')

,GDP,Infl,FFR
date,,,
1998-03-31,0.77834,0.14386,1.38
1998-06-30,0.69635,0.22873,1.38
1998-09-30,1.03077,0.36109,1.38
1998-12-31,1.37921,0.26145,1.22
1999-03-31,0.54307,0.37393,1.18
...,...,...,...
2017-03-31,0.41475,0.49969,0.18
2017-06-30,0.54594,0.25245,0.24
2017-09-30,0.54391,0.51972,0.29


# Preparing the estimation

After importing the packages and loading the data, we still need to tell pydsge how to carry out the estimation of our model. The "prep_estim" method can be used to accomplish this. It can be called without any arguments and sets-up a non-linear model by default. However, to showcase some of this functionality, we decide to specify several arguments here.

To perform the estimation, `pydsge` uses a Transposed-Ensemble Kalman Filter (TEnKF). For general information on its implementation, see the [EconSieve documentation](https://econsieve.readthedocs.io/en/latest/) , and for more details on running the filter in `pydsge` check-out the [*getting started tutorial*](https://pydsge.readthedocs.io/en/latest/getting_started.html). Again,  the default filter is non-linear, but we can opt for a linear one by setting the argument `Linear` to `True`. To choose a custom number of ensemble members for the TEnKF, set `N` to a particular number (default is 300). We can also set a specific `seed`, the default seed is `0`. To get additional information on the estimation process, we can set  `verbose` to `True`. Conveniently, this information includes an overview of the parameters’ distribution, their means and standard deviations. Moreover, if we already specified the covariance matrix of the measurement errors or want to reuse a previous result, we can load it into the `prep_estim` method by setting `Load.R` to `True`. Finally, we can turn parallelization on or off with the debug argument, which can be helpful in case any issues should arise.


In [ ]:

mod.prep_estim(N=350, seed=0, verbose=True)

After finishing our set-up, the only thing left to prepare is to filter our observed FFR for hidden states. We can simply identify the variable through `index` and, given the present context, set the measurement error to a very small value. <span style="color:Red">**@Gregor, check again** </span>

In [ ]:

mod.filter.R = mod.create_obs_cov(1e-1)
ind = mod.observables.index('FFR')
mod.filter.R[ind,ind] /= 1e1 

# Running the estimation

Now that the we have all the variables and defined the type of estimation to perform, we can turn to estimating the model. To be able to deal with very high-dimensional models, `pdygse` uses *Markov Chain Monte Carlo* (MCMC) Integration to sample from the posterior distribution. For further information on MCMC, please refer to the `emcee` [website](https://emcee.readthedocs.io/en/stable/) and the additional resources provided there. We recommend running a **Tempered Ensemble MCMC** first, by using the `tmcmc` method. Doing this is particularly valuable for high-dimensional problems, since defining the initial states of the walkers in the parameterspace in this way is a powerful tool to improve sampling. However, due to its computational efficiency, we also use it for small models such as the one we are dealing with here.

For our ensemble sampling, we can specify a variety of options. Note, `tmcmc` always requires the specification of the first four arguments, which are the i) number of steps, ii) number of walks, iii) number of temperatures, and iv) a temperature target! Here we do not want to set a target and, in turn, set `fmax = None`. Moreover, we have the option to set different "moves", i.e. coordinate updating algorithms for the walkers. As a wrapper for a lot of `emcee` functionality,  `tmcmc` can work with many different "moves" - for a list and implementation details please consult the `emcee` documentation. For using them here, specify them as a list of tuples, containing the type of move and its "weight". If no move is specified, "StretchMove" is used. For seed setting of the log probability, the user can choose between three options, here we use the seed specified in `prep_estim`<span style="color:Red">**@Gregor, check again** </span>. Finally, the states are saved in the `p0` object as a numpy array in order to later pass them to our main sampling process.

In [ ]:
fmax = None

moves = [(emcee.moves.DEMove(), 0.8), 
         (emcee.moves.DESnookerMove(), 0.2),]

p0 = mod.tmcmc(200, 200, 0, fmax, moves=moves, update_freq=100, lprob_seed='set')
mod.save()

As we can see, the output provides us with various important details. In particular, we learn that `mod.save()` saved the meta data of our model in the directory which we specified earlier in `mod.path`. This information is stored as an `.npz` file so that it is avialable even in the event of a crash and can be loaded anytime using `numpy.load()`.

We now use the initial states derived above to conduct our full Bayesian estimation. Still, initial states do not have to be specified and, unless `mcmc` can identify previous runs or estimations, the initial values of the "prior" section in the `*.yaml` are used. The default number of sampling steps is 3000, so it makes sense to allow this to run in parallel. Again, if you want to avoid this, simply set `debug` to `True`. With `tune` we can determine the size of the Markov Chain we wish to retain<span style="color:Red">**@Gregor, check again** </span>. It is important to not confuse this with the updating frequency, which only affects the number of summary statements `pydsge`reports during the estimation. Note that, like in the `tmcmc`, we choose to continue using the seed specified earlier. Lastly, the option `append` lets us store all intermediate results<span style="color:Red">**@Gregor, check again** </span>. We pickle and store the meta information of this object in the path specified earlier.

In [ ]:
mod.mcmc(p0,
         moves=moves,
         nsteps=3000,
         tune=500,
         update_freq=500,
         lprob_seed='set',
         append=True,
         debug=True,
         )
mod.save()

But, so where are our estimates? Remember that, so far, we have only drawn samples from our posterior distribution. Our converged (burnt-in) MCMC samples are currently stored in the `rank_test_sampler.h5` file created by `mcmc`. To get our parameter estimates, we now still need to draw a sample form the MCMC object. 

In [ ]:
pars = mod.get_par('posterior', nsamples=250, full=True)

Now, let's have a look at the estimated shocks. We can do this by using `extract()` which gives us the smoothed shocks. This method takes a variety of arguments, all of which have sensible default values. For example, here we specify the number of parameter draws in each verification sample to 1. <span style="color:Red">**@Gregor, check again** </span> Note that this method also takes an optional seed argument, but we here continue to use the default seed 0.  It is important to emphasise that `pysdge` seeks to separate the model's set-up (meta) data and its results. To store the Markov Chains, shocks and parameter estimates we use the `save_rdict()` method, below. 

In [ ]:
epsdf = mod.extract(pars, nsamples=1)
mod.save_rdict(epsdf)

Finally, we take a closer look at the MCMC estimation results. In particular, `mcmc_summary()` summaries the convergence behaviour of our draws from the posterior distribution.

In [ ]:
mod.mcmc_summary()

---

In [20]:
# Just for the tutorial: Cleaning the temporary directory
shutil.rmtree(output_path)